In [1]:
import re
import pandas as pd
import numpy as np

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from bertopic.representation import MaximalMarginalRelevance

import pathlib

import plotly

C:\Users\Re\Documents\GitHub\abstlyze\venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Re\Documents\GitHub\abstlyze\venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Re\Documents\GitHub\abstlyze\venv\lib\site-packages\umap\distances.

In [3]:
# Prepare data
data_set_path = pathlib.Path("dataset/fsas/fsas_full_utf8_dated.csv")
trump = pd.read_csv(str(data_set_path))
trump.abstract = trump.apply(lambda row: re.sub(r"http\S+", "", row.abstract).lower(), 1)
trump.abstract = trump.apply(lambda row: " ".join(filter(lambda x: x[0] != "@", row.abstract.split())), 1)
trump.abstract = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.abstract).split()), 1)
# trump = trump.loc[(trump.isRetweet == "f") & (trump.abstract != ""), :]
timestamps = trump.date.to_list()
abstracts = trump.abstract.to_list()

data = abstracts

arr_multi = np.array(abstracts)
print(arr_multi)

['a concept of multi valued cognitive maps is introduced in this paper the concept expands the fuzzy one however all variables and weights are not linearly ordered in the concept but are only partially ordered such an approach allows us to operate in cognitive maps with partially ordered linguistic variables directly without vague fuzzification defuzzification methods hence we may consider more subtle differences in degrees of experts uncertainty than in the fuzzy case we prove the convergence of such cognitive maps and give two simple computational examples which demonstrate using such a partially ordered uncertainty degree scale compared to the fuzzy case'
 'in this paper we investigate the fixed time synchronization for fuzzy inertial neural networks with time varying coefficients and time delays the fuzzy inertial neural networks are transformed into two forms of first order differential systems and then two kinds of different controllers of time variable are designed in these sche

In [8]:
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine',
                  # random_state=42,
                  )

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True,
                        min_samples=5)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

topic_model = BERTopic(
    embedding_model=embedding_model,  # Step 1 - Extract embeddings
    umap_model=umap_model,  # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,  # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
    min_topic_size=7  # Create more topics
)

topics, probs = topic_model.fit_transform(data)

# Use the "c-TF-IDF" strategy with a threshold
new_topics = topic_model.reduce_outliers(abstracts, topics, strategy="c-tf-idf", threshold=0.1)

# Reduce all outliers that are left with the "distributions" strategy
new_topics = topic_model.reduce_outliers(abstracts, new_topics, strategy="distributions")

topic_model.update_topics(abstracts, topics=new_topics)

In [9]:
topic_model.update_topics(data, topics=new_topics)

In [10]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,390,0_preference_decision_alternatives_making,"[preference, decision, alternatives, making, c...",[the aim of this paper is to study the integra...
1,1,459,1_learning_neural_rules_network,"[learning, neural, rules, network, algorithm, ...",[this paper proposes various methods for const...
2,2,183,2_differential_equations_solutions_differentia...,"[differential, equations, solutions, different...",[the present paper is concerned with the solut...
3,3,216,3_possibility_probability_conditional_possibil...,"[possibility, probability, conditional, possib...",[the theory of possibility described in this p...
4,4,178,4_clustering_cluster_clusters_data,"[clustering, cluster, clusters, data, algorith...",[fuzzy clustering helps to find natural vague ...
...,...,...,...,...,...
128,128,30,128_uncertainty_analysis_uncertain_engineering,"[uncertainty, analysis, uncertain, engineering...",[in many engineering practices with uncertaint...
129,129,24,129_quantum_measurements_observables_measurement,"[quantum, measurements, observables, measureme...",[quantum structures are algebraic structures l...
130,130,20,130_process_control_production_controller,"[process, control, production, controller, inc...",[the objective of this paper is to study the r...
131,131,25,131_interpolation_spline_kh_points,"[interpolation, spline, kh, points, approximat...",[in this paper we present an algorithm for rep...


In [ ]:
doc_img = topic_model.visualize_documents(abstracts, hide_annotations=True, width=910,
                                          height=585)
doc_img.show()

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(abstracts, show_progress_bar=True)
# We reduce our embeddings to 2D as it will allows us to quickly iterate later on
reduced_embeddings = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine',
                          # random_state=42,
                          ).fit_transform(embeddings)
doc_plt = topic_model.visualize_documents(abstracts, reduced_embeddings=reduced_embeddings,
                                          hide_document_hover=True, hide_annotations=True)
doc_plt.show()

In [ ]:
distance_map = topic_model.visualize_topics()
distance_map.show()